In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.impute import KNNImputer
from numpy import mean, logspace, min, max, meshgrid, linspace, c_, sqrt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD



## Data preprocessing

In [2]:

class Preprocessing:

    def __init__(self, dataframe2016, dataframe2020, index2016, index2020):
        self.df2016 = pd.read_csv(dataframe2016)
        self.df2020 = pd.read_csv(dataframe2020)
        self.df_coordinate = pd.read_csv("./data/coordinates.csv")

        self.df2016 = pd.merge(self.df2016, self.df_coordinate, how='left', on='country')
        self.df2020 = pd.merge(self.df2020, self.df_coordinate, how='left', on='country')

        self.df_index2016 = pd.read_csv(index2016)
        self.df_index2020 = pd.read_csv(index2020)

        self.output_name = None



    def replace(self):
        self.df2016 = self.df2016[self.df2016.isna().sum(axis=1) < 19]
        self.df2016.replace({'0': np.nan})

        self.df2020 = self.df2020[self.df2020.isna().sum(axis=1) < 19]
        self.df2020.replace({'0': np.nan})

    def drop(self):
         threshold2016 = self.df2016.shape[0] * 0.55
         self.df2016.dropna(axis="columns", thresh=threshold2016, inplace=True)
         self.df2016.drop("Unnamed: 0", axis=1, inplace=True)

         threshold2020 = self.df2020.shape[0] * 0.55
         self.df2020.dropna(axis="columns", thresh=threshold2020, inplace=True)
         self.df2020.drop("Unnamed: 0", axis=1, inplace=True)

    def fill_na(self):
        imputer = KNNImputer(n_neighbors=8)
        length = len(self.df2016.columns)
        self.df2016.iloc[:, 2:length] = imputer.fit_transform(self.df2016.iloc[:, 2:length])

        length = len(self.df2020.columns)
        self.df2020.iloc[:, 2:length] = imputer.fit_transform(self.df2020.iloc[:, 2:length])

    def normalization(self):
        length2016 = len(self.df2016.columns)
        self.df2016.iloc[:,1:length2016] = self.df2016.iloc[:,1:length2016].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

        length2020 = len(self.df2020.columns)
        self.df2020.iloc[:, 1:length2020] = self.df2020.iloc[:, 1:length2020].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

    def merge_index(self):
         self.df2016 = pd.merge(self.df_index2016,self.df2016, on="country", how="inner")
         self.df2020 = pd.merge(self.df_index2020,self.df2020,  on="country", how="inner")
         self.df = self.df2020.append(self.df2016)

         self.df.drop("country", axis=1, inplace=True)
         self.df.drop("Flood occurrence (WRI) (-)", axis=1, inplace=True)

    def write_2020data(self):
        self.output_name2020 = "./data/final data/" + "processed_data_2020" + ".csv"
        self.df2020.to_csv(self.output_name2020)

    def save(self, output_name, norm=True, write=True):
        self.replace()
        self.drop()
        self.fill_na()
        if norm:
            self.normalization()
        self.write_2020data()
        self.merge_index()
        if write:
            self.output_name = "./data/final data/" + output_name + ".csv"
            self.df.to_csv(self.output_name)



    def get_dataframe(self):
        return pd.read_csv(self.output_name)

In [ ]:
data2016 = Preprocessing("./data/raw data/2016raw.csv","./data/raw data/2020raw.csv","./data/2016APindex.csv","./data/2020APindex.csv")
data2016.save("newData", norm=True)

## Model

In [ ]:

def split(x, y):
    return train_test_split(x, y, test_size=0.33, random_state=0)


class TreeModelBuilder:

    def __init__(self, dataframe):
        self.dataframe = pd.read_csv(dataframe)
        self.df = self.dataframe.values
        self.x = self.df[:, 2:]
        self.y = self.df[:, 1].astype('float')
        self.x_train, self.x_test, self.y_train, self.y_test = split(self.x, self.y)

    def __building_model(self, x_train, y_train, model_type, number=1000):
        if model_type == "RD":
            classifier = RandomForestClassifier(n_estimators=number, random_state=0, n_jobs=-1)
        elif model_type == "Bagging":
            classifier = BaggingClassifier(n_estimators=1000, random_state=0).fit(x_train, y_train)
        elif model_type == "ADA":
            classifier = AdaBoostClassifier(n_estimators=1000, learning_rate=1).fit(x_train, y_train)
        else:
            classifier = GradientBoostingClassifier(n_estimators=number, learning_rate=1, max_depth=1, random_state=0)
        return classifier.fit(x_train, y_train)

    

    def __rebuild_model(self, model, model_type, number=0.05):
        rebuild = SelectFromModel(model, threshold=number).fit(self.x_train, self.y_train)
        x_important_train, x_important_test = rebuild.transform(self.x_train), rebuild.transform(self.x_test)
        rebuild = self.__building_model(x_important_train, self.y_train, model_type=model_type, number=1000)
        prediction = rebuild.predict(x_important_test)
        return rebuild, prediction

    def __get_score(self, model):
        return model.score(self.x_test, self.y_test)

    def __get_accuracy(self, prediction):
        return accuracy_score(self.y_test, prediction)

    def important_features(self, model):
        results = model.feature_importances_
        for i in range(len(self.df.columns[2:])):
            print(self.df.columns[i] + results[i] + "\n")

    def save_plot(self, model):
        plt.subplots(2, 2)
        for i in range(0, 5):
            fig = plt.figure(figsize=(20, 20))
            tree.plot_tree(model.estimators_[i], feature_names=self.dataframe.columns[2:], filled=True, fontsize=10)
            plt.show()

    def decision_boundary(self):
        model = self.__building_model(x_train=self.x_train, y_train=self.x_train, model_type="RD")
        x_train_reduced = TruncatedSVD(n_components=2, random_state=0).fit_transform(self.x_train)
        prediction = model.predict(self.y_train)

        x2d_x_min, x2d_x_max = min(x_train_reduced[:, 0]), max(x_train_reduced[:, 0])
        x2d_y_min, x2d_y_max = min(x_train_reduced[:, 1]), max(x_train_reduced[:, 1])
        xx, yy = meshgrid(linspace(x2d_x_min, x2d_x_max, 100), linspace(x2d_y_min, x2d_y_max, 100))

        background_model = KNeighborsClassifier(n_neighbors=5).fit(x_train_reduced, prediction)
        voronoi_background = background_model.predict(c_[xx.ravel(), yy.ravel()]).reshape((100, 100))

        plt.contourf(xx, yy, voronoi_background)
        plt.scatter(x_train_reduced[:, 0], x_train_reduced[:, 1], c=prediction)
        plt.show()

    def get_model(self, model_type, number=0.05, accuracy=False,rebuit=False):
        if model_type == "Boost":
            number = 0.01
        model = self.__building_model(self.x_train, self.y_train, model_type=model_type)
        if model_type != "Bagging" and model_type != "ADA":
            rebuild_model = self.__rebuild_model(model, model_type=model_type, number=number)
            if accuracy:
                return [self.__get_score(model), self.__get_accuracy(rebuild_model[1])]
            else:
                return [model, rebuild_model[0]]
        else:
            if accuracy:
                prediction = model.predict(self.x_test)
                return [self.__get_score(model), self.__get_accuracy(prediction)]
            else:
                return [model]


    def getrfrebuild(self,x=[],name=False):
        model=RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1).fit(self.x_train, self.y_train)
        rebuild1 = SelectFromModel(model, threshold=0.05).fit(self.x_train, self.y_train)
        x_important_train, x_important_test = rebuild1.transform(self.x_train), rebuild1.transform(self.x_test)
        rebuild = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1).fit(x_important_train, self.y_train)
        featureindex=rebuild1.get_support(indices=True)
        featurename=self.dataframe.columns[featureindex+2]
        if len(x)!=0:
            prediction = rebuild.predict(x)
        if name:
            return featurename
        else:
            return prediction
    

        
       


class SupportVectorMachineBuilder:

    def __init__(self, dataframe):
        self.df = pd.read_csv(dataframe)
        self.y = self.df['water security index']
        self.x = self.df.iloc[:, 2:]
        self.x_train, self.x_test, self.y_train, self.y_test = split(self.x, self.y)
        self.model = None

    def __build_model(self, c=None, gamma=None, tuning=False):
        if tuning:
            clf = svm.SVC(decision_function_shape='ovo', C=c, gamma=gamma)
        else:
            clf = svm.SVC(decision_function_shape='ovo')
        model = clf.fit(self.x_train, self.y_train)
        return model

    def __tuning_model(self):
        pipe = Pipeline([("svc", self.__build_model())])
        grid_parameters = {
            'svc__C': [2 ** x for x in range(-5, 13)],
            'svc__gamma': [2 ** x for x in range(-12, 4)]
        }
        grid = GridSearchCV(pipe, param_grid=grid_parameters,
                            cv=3, scoring=make_scorer(f1_score, average="weighted"), n_jobs=2,
                            return_train_score=True, verbose=3)
        grid.fit(self.x_train, self.y_train)
        best_parameters = [grid.best_params_.get("svc__C"), grid.best_params_.get("svc__gamma")]
        rebuild_model = self.__build_model(best_parameters[0], best_parameters[1], tuning=True)
        return rebuild_model

    def get_accuracy(self):
        accuracy_score_not_tuned = self.__build_model().score(self.x_test, self.y_test)
        accuracy_score_tuned = self.__tuning_model().score(self.x_test, self.y_test)
        return str("Results of model: " + str(accuracy_score_not_tuned) + "\n" +
                   "Results of tuned model:" + str(accuracy_score_tuned))


class RidgeLassoBuilder:

    def __init__(self, dataframe, alpha):
        self.df = pd.read_csv(dataframe).values
        self.x = self.df[:, 2:self.df.shape[1]]
        self.y = self.df[:, 1].astype('float')
        self.x_train, self.x_test, self.y_train, self.y_test = split(self.x, self.y)
        self.alpha = alpha

    def __building_model(self):
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=0)
        if self.alpha == 0:
            model = RidgeCV(alphas=logspace(-4, -0.5, 30), cv=cv)
        else:
            model = LassoCV(alphas=logspace(-4, -0.5, 30), cv=cv)
        model = model.fit(self.x_train, self.y_train)
        scores = cross_val_score(model, self.x_train, self.y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
        prediction = [round(x, 0) for x in list(model.predict(self.x_test))]
        return [mean(scores), accuracy_score(self.y_test, prediction)]

    def get_plots_ridge_alpha(self):
        alphas = logspace(-10, -2, 30)
        coefs = []
        for a in alphas:
            ridge = Ridge(alpha=a, fit_intercept=False)
            ridge.fit(self.x_train, self.y_train)
            coefs.append(ridge.coef_)

        plot = plt.gca()
        plot.plot(alphas, coefs)
        plot.set_xscale('log')
        plot.set_xlim(plot.get_xlim()[::-1])
        plt.xlabel('Alpha')
        plt.ylabel('Weights')
        plt.title('Ridge coefficients as a function of the regularization')
        plt.axis('tight')
        plt.show()

    def get_plots_ridge_cv(self):
        model = Lasso(random_state=0, max_iter=10000)
        alphas = logspace(-4, -0.5, 30)
        tuned_parameters = [{'alpha': alphas}]
        model = GridSearchCV(model, tuned_parameters, cv=5, refit=False)
        model.fit(self.x_train, self.y_train)
        scores = model.cv_results_['mean_test_score']
        scores_std = model.cv_results_['std_test_score']
        std_error = scores_std / sqrt(5)

        plt.figure().set_size_inches(8, 6)
        plt.semilogx(alphas, scores)
        plt.semilogx(alphas, scores + std_error, 'b--')
        plt.semilogx(alphas, scores - std_error, 'b--')
        plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)
        plt.ylabel('CV score +/- std error')
        plt.xlabel('alpha')
        plt.axhline(max(scores), linestyle='--', color='.5')
        plt.xlim([alphas[0], alphas[-1]])
        plt.show()

    def get_accuracy(self):
        results = self.__building_model()
        return str("Prediction " + str(results[1]) + "\n" +
                   "Accuracy based CV:" + str(results[0]))



## Model 2025

In [ ]:

#predict the index of 2025
mod1=TreeModelBuilder('./data/final data/newData.csv')

featurename=mod1.getrfrebuild(name=True)
print(featurename)


print(mod1.get_model("Boost", accuracy=True))
print(mod1.get_model("Bagging", accuracy=True))
print(mod1.get_model("ADA", accuracy=True))
mod2 = SupportVectorMachineBuilder('./data/final data/newData.csv')
print(mod2.get_accuracy())
mod3 = RidgeLassoBuilder('./data/final data/newData.csv', 0)
print(mod3.get_accuracy())
print(mod1.get_model("RD", accuracy=True))




all2020=pd.read_csv("./data/final data/processed_data_2020.csv")
x2020=all2020[featurename]
y=mod1.getrfrebuild(x=x2020,name=False)
all2020["water security index"]=y
all2020.to_csv("allindex2020.csv")


#====================
#Get all the variables of 2025


df=pd.read_csv('./data/newaquastat.csv')
df=df.dropna(axis=0)

def extract(name):
    data=df.loc[df['Variable Name']==name]
    return data

def get2025(name):
    df_AWW=df.loc[df['Variable Name']==name]
    area=df_AWW['Area']
    area=list(set(area))
    df_AWW2025=pd.DataFrame(columns=('Area',name))
    for i in range(0,len(area)):
        df_areai=df_AWW.loc[df_AWW['Area']==area[i]]
        x=df_areai['Year']
        y=df_areai['Value']
        p=np.poly1d(np.polyfit(x,y,1))
        predict=p(2025)
        df_AWW2025=df_AWW2025.append(pd.DataFrame({'Area':[area[i]],name:[predict]}))
    return df_AWW2025


def mergeall(vv):
    length=len(vv)
    result=pd.merge(get2025(vv[0]),get2025(vv[1]),on=['Area'])
    for i in range(2,length):
        result=pd.merge(result,get2025(vv[i]),on=['Area'])
    return result


def getfromother(name11):
    df_PA65=pd.read_csv("./data/other/"+ name11+".csv",header=None)
    df_PA65=df_PA65.dropna(axis=0)
    x=pd.to_numeric(df_PA65.iloc[0,3:])
    df_PA2025=pd.DataFrame(columns=('Area',name11))
    for i in range(1,df_PA65.shape[0]):
        y=pd.to_numeric(df_PA65.iloc[i,3:])
        p=np.poly1d(np.polyfit(x,y,4))
        predict=p(2025)
        df_PA2025=df_PA2025.append(pd.DataFrame({'Area':[df_PA65.iloc[i,0]],name11:[predict]}))
    return df_PA2025



### Generate the results of the two models

In [ ]:


name1='GDP per capita (current US$/inhab)'
name2='Agriculture, value added (% GDP) (%)'
name3='Human Development Index (HDI) [highest = 1] (-)'
name4='Agricultural water withdrawal as % of total water withdrawal (%)'
name5='Total population with access to safe drinking-water (JMP) (%)'
name6='Urban population with access to safe drinking-water (JMP) (%)'
name7="Mortality rate, infant (per 1,000 live births)"
name8="Net official development assistance and official aid received (current US$)"

vv=[name1,name2,name3,name4,name5,name6]
result1=mergeall(vv)
result1=pd.merge(result1,getfromother(name8),on=['Area'])
result1=pd.merge(result1,getfromother(name7),on=['Area'])

result1.to_csv('variable_2025.csv')

#using the model to predict the 2025 indexes.





newx=pd.read_csv("variable_2025.csv")
xx=newx.values[:,2:]
yy=mod1.getrfrebuild(x=xx,name=False)
newx["water security index"]=yy
newx.to_csv("allindex2025.csv")


## Analysis about index and population

In [ ]:
#This analysis is to use the final output to gain some knowledge about the relationship with population related variables and index.
sns.set_theme(style="dark",palette=sns.color_palette("husl",2))
data=pd.read_csv("allindex2020.csv",index_col=[0])
data.isnull().sum()
data=data.dropna()

# Group index into two classes

data['water security index']=data['water security index'].map({1:0,2:0,3:1,4:1})


# To exclude the possibility of water stress to be the respond variable by looking at the correlation between it and the others.
listpop=['Rural population (1000 inhab)', 
           'Urban population (1000 inhab)',
           'Population density (inhab/km2)',
           'Urban population with access to safe drinking-water (JMP) (%)',
           'Rural population with access to safe drinking-water (JMP) (%)',
           'Total population with access to safe drinking-water (JMP) (%)',
           'population ages0-14',
           'Population ages 65 and above (_ of total population)']
data[listpop].corrwith(data['SDG 6.4.2. Water Stress (%)'])


In [ ]:
# To see some difference from the mean value of 0 and 1 classes

data[['Rural population (1000 inhab)', 
           'Urban population (1000 inhab)',
           'Population density (inhab/km2)',
           'Urban population with access to safe drinking-water (JMP) (%)',
           'Rural population with access to safe drinking-water (JMP) (%)',
           'Total population with access to safe drinking-water (JMP) (%)',
           'population ages0-14',
           'Population ages 65 and above (_ of total population)','water security index'
     ]].groupby('water security index').mean().T

In [ ]:

for x in listpop:
    plt.figure(figsize=(9,6))
    g1=data[data['water security index']==0][x].values
    g2=data[data['water security index']==1][x].values
    
    plt.hist(g1,bins=50,label="index=0",color='green',alpha=0.5)
    plt.hist(g2,bins=50,label="index=1",color='orange',alpha=0.5)
    
    # Two dot lines represent the mean of 0 and 1 group
    g1_mean=g1.mean()
    g2_mean=g2.mean()
    plt.axvline(g1_mean,ls='--',c='green',linewidth=1)
    plt.axvline(g2_mean,ls='--',c='orange',linewidth=1)
    
    plt.legend()
    plt.title("Historgram by Different Index Group\n{}".format(x),fontsize=14)
    plt.show()

In [ ]:
# Use t.test to see the significance of variables

for col in listpop:
    
    # Get the values of 0 and 1 seperately
    g1=data[data['water security index']==0][col].values
    g2=data[data['water security index']==1][col].values

    p_levene=stats.levene(g1, g2)[1]
    if p_levene>0.05:
        statistic,pvalue_ttest=stats.ttest_ind(g1, g2, equal_var = True)
    else:
        statistic,pvalue_ttest=stats.ttest_ind(g1, g2, equal_var = False)
        
    print("{:<70}: ttest- statistic={:>10.5f};pvalue={:>10.5f}".format(col,statistic,pvalue_ttest))

+ The last five variables has significant influence on index classification

In [ ]:
# Use the feature of random forest model to output feature importance
# So here use random forest to build a classification model where x is the features and y is index

from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(random_state=1)

x_cols=listpop
        
x=data[x_cols]
y=data['water security index']

classifier.fit(x,y)
feature_importances=classifier.feature_importances_

df_feature_importances=pd.DataFrame(zip(x_cols,feature_importances),columns=["features","importance"])
df_feature_importances

In [ ]:
# Visualize the importance

plt.figure(figsize=(16,9))
sns.barplot(x="features",y="importance",data=df_feature_importances)
plt.xticks(rotation=-90)